Load in required libraries, below.

In [1]:
# data 
import pandas as pd 
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

%matplotlib inline

## SageMaker Resources

The below cell stores the SageMaker session and role (for creating estimators and models), and creates a default S3 bucket. After creating this bucket, you can upload any locally stored data to S3.

In [4]:
# sagemaker
import boto3
import sagemaker
from sagemaker import get_execution_role

In [5]:
# SageMaker session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# default S3 bucket
bucket = sagemaker_session.default_bucket()

In [ ]:
!wget https://da-youtube-ml.s3.eu-central-1.amazonaws.com/wendy-cnn/frames/wendy_cnn_frames_data.zip
!unzip wendy_cnn_frames_data.zip -d wendy_cnn_frames_data 
!rm wendy_cnn_frames_data.zip

# upload to S3
input_data = sagemaker_session.upload_data(path='wendy_cnn_frames_data', bucket=bucket, key_prefix=prefix)
print(input_data)

In [ ]:
The next cell runs the above function to create a `train.csv` file in a specified directory.

The next cell runs the above function to create a `train.csv` file in a specified directory.

Check that you've uploaded the data, by printing the contents of the default bucket.

In [1]:
!pygmentize letsplay_classifier/model.py

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, models, transforms
from torchvision.models.vgg import make_layers, VGG, cfgs

class VGGLP(VGG):

    def __init__(self, num_classes):
        super(VGGLP, self).__init__(make_layers(cfgs['D'], batch_norm=True))
        num_features = self.classifier[6].in_features
        features = list(self.classifier.children())[:-1]  # Remove last layer
        features.extend([nn.Linear(num_features, num_classes)])  # Add our layer with 4 outputs
        self.classifier = nn.Sequential(*features)  # Replace the model classifier


In [2]:
!pygmentize letsplay_classifier/train.py

from __future__ import print_function # future proof
import argparse
import time
import copy
import os
import json

import pandas as pd
LOCAL = 1
# pytorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data

import torch
import torch.nn as nn
import torch.optim as optim

from torch.autograd import Variable
import torchdata as td
import torchvision
from torchvision import datasets, models, transforms

# import model
from model import VGGLP


def model_fn(model_dir):
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = VGGLP(model_info.num_classes)

    # Load the stored mode

### EXERCISE: Create a PyTorch Estimator

You've had some practice instantiating built-in models in SageMaker. All estimators require some constructor arguments to be passed in. When a custom model is constructed in SageMaker, an **entry point** must be specified. The entry_point is the training script that will be executed when the model is trained; the `train.py` function you specified above! 

See if you can complete this task, instantiating a PyTorch estimator, using only the [PyTorch estimator documentation](https://sagemaker.readthedocs.io/en/stable/sagemaker.pytorch.html) as a resource. It is suggested that you use the **latest version** of PyTorch as the optional `framework_version` parameter.

#### Instance Types

It is suggested that you use instances that are available in the free tier of usage: `'ml.c4.xlarge'` for training and `'ml.t2.medium'` for deployment.

In [ ]:
# import a PyTorch wrapper
from sagemaker.pytorch import PyTorch

# specify an output path
# prefix is specified above
output_path = 's3://{}/{}'.format(bucket, prefix)

# instantiate a pytorch estimator
estimator = PyTorch(entry_point='train.py',
                    source_dir='letsplay_classifier', # this should be just "source" for your code
                    role=role,
                    framework_version='1.0',
                    train_instance_count=1,
                    train_instance_type='ml.c4.xlarge',
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'img-width': 128,
                        'img-height': 72,
                        'batch-size': 16,
                        'epochs': 10
'
                    })

In [3]:
## Train the Estimator

After instantiating your estimator, train it with a call to `.fit()`. The `train.py` file explicitly loads in `.csv` data, so you do not need to convert the input data to any other format.

ModuleNotFoundError: No module named 'sagemaker'

## Train the Estimator

After instantiating your estimator, train it with a call to `.fit()`. The `train.py` file explicitly loads in `.csv` data, so you do not need to convert the input data to any other format.

In [13]:
%%time 
# train the estimator on S3 training data
estimator.fit({'train': input_data})

INFO:sagemaker:Creating training-job with name: sagemaker-pytorch-2019-03-12-03-20-56-668


2019-03-12 03:20:56 Starting - Starting the training job...
2019-03-12 03:21:00 Starting - Launching requested ML instances......
2019-03-12 03:22:00 Starting - Preparing the instances for training......
2019-03-12 03:23:24 Downloading - Downloading input data
2019-03-12 03:23:24 Training - Training image download completed. Training in progress.
2019-03-12 03:23:24 Uploading - Uploading generated training model
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-03-12 03:23:14,349 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-03-12 03:23:14,352 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-03-12 03:23:14,368 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-03-12 03:23:15,772 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-03-12 03:23:16,040 sagemaker-containers INF

## Create a Trained Model

PyTorch models do not automatically come with `.predict()` functions attached (as many Scikit-learn models do, for example) and you may have noticed that you've been give a `predict.py` file. This file is responsible for loading a trained model and applying it to passed in, numpy data. When you created a PyTorch estimator, you specified where the training script, `train.py` was located. 

> How can we tell a PyTorch model where the `predict.py` file is?

Before you can deploy this custom PyTorch model, you have to take one more step: creating a `PyTorchModel`. In earlier exercises you could see that a call to `.deploy()` created both a **model** and an **endpoint**, but for PyTorch models, these steps have to be separate.

### EXERCISE: Instantiate a `PyTorchModel`

You can create a `PyTorchModel` (different that a PyTorch estimator) from your trained, estimator attributes. This model is responsible for knowing how to execute a specific `predict.py` script. And this model is what you'll deploy to create an endpoint.

#### Model Parameters

To instantiate a `PyTorchModel`, ([documentation, here](https://sagemaker.readthedocs.io/en/stable/sagemaker.pytorch.html#sagemaker.pytorch.model.PyTorchModel)) you pass in the same arguments as your PyTorch estimator, with a few additions/modifications:
* **model_data**: The trained `model.tar.gz` file created by your estimator, which can be accessed as `estimator.model_data`.
* **entry_point**: This time, this is the path to the Python script SageMaker runs for **prediction** rather than training, `predict.py`.


In [14]:
# importing PyTorchModel
from sagemaker.pytorch import PyTorchModel

# Create a model from the trained estimator data
# And point to the prediction script
model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='1.0',
                     entry_point='predict_fromfile.py',
                     source_dir='letsplay_classifier')


### EXERCISE: Deploy the trained model

Deploy your model to create a predictor. We'll use this to make predictions on our test data and evaluate the model.

In [15]:
%%time
# deploy and create a predictor
predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

INFO:sagemaker:Creating model with name: sagemaker-pytorch-2019-03-12-03-24-09-384
INFO:sagemaker:Creating endpoint with name sagemaker-pytorch-2019-03-12-03-24-09-384


---------------------------------------------------------------------------------------!CPU times: user 591 ms, sys: 62.9 ms, total: 654 ms
Wall time: 7min 21s


In [ ]:
from predict_fromfile import model_fn, predict_fn, input_fn, output_fn
import argparse
import os
import json
import torch
from PIL import Image

import torch.nn as nn

index = 0
loss_test = 0
acc_test = 0
count = 0
cur_data_dir = 'wendy_cnn_frames_data'
dirs = sorted(os.listdir(cur_data_dir))
for dir in dirs:
    labels = torch.empty(1, dtype=int)
    labels[0] = index
    print(labels)
    curr_img_dir = os.path.join(args.data_dir, dir)
    images = os.listdir(curr_img_dir)
    for image in images:
        curr_img = os.path.join(curr_img_dir, image)

        with open(curr_img, 'rb') as f:
            image_data = Image.open(f)
           
            prediction_from_ep = predictor.predict(image_data)  
            prediction = torch.FloatTensor(prediction_from_ep).unsqueeze(0)

            _, preds = torch.max(prediction.data, 1)
            loss = criterion(prediction, labels)

            loss_test += loss.data
            acc_test += torch.sum(preds == labels.data)
            count += 1
            avg_loss = torch.true_divide(loss_test, count)
            avg_acc = torch.true_divide(acc_test, count)
            print("{} processed".format(count))
            print("Avg loss (test): {:.4f}".format(avg_loss))
            print("Avg acc (test): {:.4f}".format(avg_acc))
    index += 1

---
## Evaluating Your Model

Once your model is deployed, you can see how it performs when applied to the test data.

The provided function below, takes in a deployed predictor, some test features and labels, and returns a dictionary of metrics; calculating false negatives and positives as well as recall, precision, and accuracy.

In [16]:

import torch
import os
import json
import time
from letsplay_classifiers.train import get_data_loaders
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.optim as optim

def eval_predictor(predictor, dataloaders):
    
    test_batches = len(dataloaders['val'])
    print("Evaluating model")
    print('-' * 10)

    for i, data in enumerate(dataloaders['val']):
        if i % 100 == 0:
            print("\rTest batch {}/{}".format(i, test_batches), end='', flush=True)


dataloaders, dataset_sizes, class_names = get_data_loaders(
    img_dir='wendy-cnn-data', 256, 256, 8 )

eval_predictor(predictor) 


### Test Results

The cell below runs the `evaluate` function. 

The code assumes that you have a defined `predictor` and `X_test` and `Y_test` from previously-run cells.

In [17]:
# get metrics for custom predictor
metrics = evaluate(predictor, X_test, Y_test, True)

predictions  0.0  1.0
actuals              
0             53   18
1             11   68

Recall:     0.861
Precision:  0.791
Accuracy:   0.807



## Delete the Endpoint

Finally, I've add a convenience function to delete prediction endpoints after we're done with them. And if you're done evaluating the model, you should delete your model endpoint!

In [18]:
# Accepts a predictor endpoint as input
# And deletes the endpoint by name
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))

In [19]:
# delete the predictor endpoint 
delete_endpoint(predictor)

Deleted sagemaker-pytorch-2019-03-12-03-24-09-384


## Final Cleanup!

* Double check that you have deleted all your endpoints.
* I'd also suggest manually deleting your S3 bucket, models, and endpoint configurations directly from your AWS console.

You can find thorough cleanup instructions, [in the documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html).

---
# Conclusion

In this notebook, you saw how to train and deploy a custom, PyTorch model in SageMaker. SageMaker has many built-in models that are useful for common clustering and classification tasks, but it is useful to know how to create custom, deep learning models that are flexible enough to learn from a variety of data.